In [3]:
from functools import lru_cache
import random
import numpy as np
import pygame
from pprint import pprint as print

GENERATIONS = 200
POPULATION_SIZE = 15
MUTATION_RATE = 0.01


# Define the Task class
class Task:
    def __init__(self, task_id: int, duration: int, priority: int):
        self.id = task_id
        self.duration = duration
        self.priority = priority

    def __repr__(self):
        return f"Task(id={self.id}, duration={self.duration}, priority={self.priority})"


# Define the Resource class
class Resource:
    def __init__(self, resource_id: int, capacity: int):
        self.id = resource_id
        self.capacity = capacity

    def __repr__(self):
        return f"Resource(id={self.id}, capacity={self.capacity})"


# Chromosome representation
class Chromosome:
    def __init__(self, gene: list[int], fitness: int = 0):
        self.gene = gene  # Gene is a list of task assignments to resources
        self.fitness = fitness

    def __lt__(self, other: "Chromosome"):
        return self.fitness < other.fitness


# Fitness function
def calculate_fitness(
    chromosome: Chromosome, tasks: list[Task], resources: list[Resource]
) -> float:
    # Calculate total execution time and priority score
    resource_times: dict[int, float] = {r.id: 0 for r in resources}
    priority_score = 0

    for task_idx, resource_id in enumerate(chromosome.gene):
        task = tasks[task_idx]
        resource_times[resource_id] += task.duration
        priority_score += task.priority

    makespan = max(resource_times.values())
    load_balance = np.std(list(resource_times.values()))

    # Objective: Minimize makespan and load balance, maximize priority score
    fitness = (
        (1 / makespan) + (1 / (1 + load_balance)) + (priority_score / (len(tasks) * 5))
    )
    return fitness


# Selection operator
def selection(population: list[Chromosome]) -> tuple[Chromosome, Chromosome]:
    # Using tournament selection
    tournament_size = 3
    # select random chromosomes and return the best two
    selected = random.sample(population, tournament_size)
    selected.sort(reverse=True)
    return (selected[0], selected[1])


# Crossover operator
def crossover(parent1: Chromosome, parent2: Chromosome) -> Chromosome:
    crossover_point = random.randint(1, len(parent1.gene) - 2)
    child_gene = parent1.gene[:crossover_point] + parent2.gene[crossover_point:]
    return Chromosome(gene=child_gene)


# Mutation operator
def mutation(chromosome: Chromosome, num_resources: int, mutation_rate: float = 0.01):
    for i in range(len(chromosome.gene)):
        if random.random() < mutation_rate:
            chromosome.gene[i] = random.randint(0, num_resources - 1)


# Initialize Pygame
pygame.init()
WIDTH, HEIGHT = 3300, 800
SCREEN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Task Scheduling Visualization")
FONT = pygame.font.SysFont("Arial", 16)


# Function to create the schedule with start times for visualization
def create_schedule(
    chromosome: Chromosome, tasks: list[Task], resources: list[Resource]
) -> dict[int, list[dict]]:
    schedule = {r.id: [] for r in resources}
    resource_end_times = {r.id: 0 for r in resources}

    for task_idx, resource_id in enumerate(chromosome.gene):
        task = tasks[task_idx]
        start_time = resource_end_times[resource_id]
        finish_time = start_time + task.duration
        schedule[resource_id].append(
            {
                "Task": task,
                "Start": start_time,
                "Finish": finish_time,
            }
        )
        resource_end_times[resource_id] = finish_time

    return schedule


@lru_cache(maxsize=None)
def create_color(_task_id: str) -> tuple[int, int, int]:
    return (random.randint(20, 255), random.randint(30, 255), random.randint(40, 255))


# Function to draw the schedule using Pygame
def draw_schedule(schedule, generation, best_fitness: float):
    SCREEN.fill((255, 255, 255))
    resource_height = HEIGHT // (len(schedule) + 1)
    max_time = max(
        [task_info["Finish"] for tasks in schedule.values() for task_info in tasks]
    )
    time_scale = WIDTH / (max_time + 1)

    for idx, (resource_id, tasks_info) in enumerate(schedule.items()):
        y = (idx + 1) * resource_height
        resource_label = FONT.render(f"Resource {resource_id}", True, (0, 0, 0))
        SCREEN.blit(resource_label, (5, y - resource_height // 2))

        for task_info in tasks_info:
            task = task_info["Task"]
            start = task_info["Start"]
            duration = task.duration
            x = start * time_scale
            width = duration * time_scale

            task_id = f"T{task.id}"
            color = create_color(task_id)

            pygame.draw.rect(
                SCREEN,
                color,
                (x + 100, y - resource_height // 2, width, resource_height - 10),
                0,
            )
            task_label = FONT.render(task_id, True, (255, 255, 255))
            SCREEN.blit(task_label, (x + 105, y - resource_height // 2 + 5))

    # Display generation and fitness
    gen_text = FONT.render(f"Generation: {generation}", True, (0, 0, 0))
    fitness_text = FONT.render(f"Best Fitness: {best_fitness:.4f}", True, (0, 0, 0))
    SCREEN.blit(gen_text, (WIDTH - 200, 10))
    SCREEN.blit(fitness_text, (WIDTH - 200, 30))

    pygame.display.update()


# Genetic Algorithm function with Pygame visualization
def genetic_algorithm(
    tasks: list[Task],
    resources: list[Resource],
    population_size: int = 50,
    generations: int = 100,
):
    num_tasks = len(tasks)
    num_resources = len(resources)

    # Initialize population
    population: list[Chromosome] = []

    for _ in range(population_size):
        gene = [random.randint(0, num_resources - 1) for _ in range(num_tasks)]
        chromosome = Chromosome(gene)
        chromosome.fitness = calculate_fitness(chromosome, tasks, resources)
        population.append(chromosome)

    # Evolution process
    for gen in range(generations):
        new_population: list[Chromosome] = []

        # Elitism: Keep the best chromosome
        population.sort(reverse=True)
        new_population.append(population[0])

        while len(new_population) < population_size:
            # Selection
            parent1, parent2 = selection(population=population)

            # Crossover
            child = crossover(parent1=parent1, parent2=parent2)

            # Mutation
            mutation(chromosome=child, num_resources=num_resources)

            # Calculate fitness
            child.fitness = calculate_fitness(
                chromosome=child, tasks=tasks, resources=resources
            )
            new_population.append(child)

        population = new_population

        best_chromosome = max(population, key=lambda c: c.fitness)

        # Handle Pygame events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return best_chromosome

        # Create schedule with start and finish times
        schedule = create_schedule(best_chromosome, tasks, resources)

        # Draw the schedule
        draw_schedule(schedule, gen, best_chromosome.fitness)

        # Control the speed of visualization
        pygame.time.delay(300)

    pygame.time.delay(10000)
    pygame.quit()
    return best_chromosome


# Main function
def main():
    num_tasks = 100
    num_resources = 5
    resource_capacity = 10

    tasks = generate_tasks(num_tasks)
    resources = generate_resources(num_resources, resource_capacity)

    print("Running Genetic Algorithm...")
    print(f"Tasks: {tasks}")
    print(f"Resources: {resources}")

    best_chromosome = genetic_algorithm(
        tasks=tasks,
        resources=resources,
        population_size=POPULATION_SIZE,
        generations=GENERATIONS,
    )

    print(f"Best Fitness = {best_chromosome.fitness:.4f}")
    print(f"Best Gene = {best_chromosome.gene}")


if __name__ == "__main__":
    # Generate sample tasks
    def generate_tasks(num_tasks: int):
        tasks = []
        for task_id in range(num_tasks):
            # Task duration between 1 and 10 units
            duration = random.randint(1, 10)
            # Priority between 1 (lowest) and 5 (highest)
            priority = random.randint(1, 5)
            tasks.append(Task(task_id=task_id, duration=duration, priority=priority))
        return tasks

    # Generate sample resources
    def generate_resources(num_resources: int, capacity: int):
        return [Resource(resource_id=i, capacity=capacity) for i in range(num_resources)]

    main()

ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default


'Running Genetic Algorithm...'
('Tasks: [Task(id=0, duration=8, priority=5), Task(id=1, duration=10, '
 'priority=5), Task(id=2, duration=8, priority=2), Task(id=3, duration=8, '
 'priority=2), Task(id=4, duration=9, priority=2), Task(id=5, duration=1, '
 'priority=5), Task(id=6, duration=3, priority=4), Task(id=7, duration=2, '
 'priority=3), Task(id=8, duration=5, priority=3), Task(id=9, duration=8, '
 'priority=3), Task(id=10, duration=1, priority=2), Task(id=11, duration=5, '
 'priority=5), Task(id=12, duration=9, priority=2), Task(id=13, duration=1, '
 'priority=3), Task(id=14, duration=9, priority=1), Task(id=15, duration=10, '
 'priority=3), Task(id=16, duration=8, priority=5), Task(id=17, duration=3, '
 'priority=5), Task(id=18, duration=9, priority=2), Task(id=19, duration=8, '
 'priority=3), Task(id=20, duration=2, priority=4), Task(id=21, duration=8, '
 'priority=1), Task(id=22, duration=7, priority=1), Task(id=23, duration=2, '
 'priority=4), Task(id=24, duration=8, priorit